In [1]:
%pip install groq
%pip install nltk

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from groq import Groq

In [3]:
os.environ['GROQ_API_KEY'] = "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"

client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

### Download colocado acima para diminuir tempo de execução do código

In [4]:
nltk.download('punkt', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)

True

### Classe Agent gerencia a interação com o modelo

In [5]:
class Agent:
    def __init__(self, client: Groq, system: str = "") -> None:
        self.client = client
        self.system = system
        self.messages: list = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message=""):
        # Mantém histórico limitado (últimas 10 mensagens)
        if len(self.messages) > 10:
            self.messages = [self.messages[0]] + self.messages[-9:]
        if message:
            self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        try:
            # Chamada à API com tratamento de erros
            completion = self.client.chat.completions.create(
                model="llama3-70b-8192",
                messages=self.messages
            )
            if not completion.choices:
                raise ValueError("Resposta vazia da API")
            return completion.choices[0].message.content
        except Exception as e:
            print(f"Erro na API: {e}")
            return f"Erro: {e}"
    
    def reset(self):
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": self.system})

### Função de análise linguística usando NLTK

In [6]:
def class_words(frase):
    # Tokenização e tagging gramatical
    tokens = word_tokenize(frase) # Separa a frase em palavras
    tags = pos_tag(tokens) # Classifica gramaticalmente cada palavra
    return str(tags)

### Sistema principal de classificação de frases

In [7]:
def classifie_sentence(sentence):
    # Define um prompt estruturado no formato ReAct
    system_prompt = """
    You run in a loop of Thought, Action, PAUSE, Observation.
    At the end of the loop you output an Answer.
    Use Thought to describe your thoughts about the question you have been asked.
    Use Action to run one of the actions available to you - then return PAUSE.
    Observation will be the result of running those actions.

    Your available actions are:
    class_words:
    e.g. class_words(frase)
    returns a list with the words and their grammatical class

    Example session:
    Frase: The black cat jumped over the tall brick wall.
    Thought: I need to analyze the grammatical structure of this sentence.
    Action: class_words(The black cat jumped over the tall brick wall)
    PAUSE

    Observation: [('The', 'DT'), ('black', 'JJ'), ('cat', 'NN'), ('jumped', 'VBD'), ('over', 'IN'), ('the', 'DT'), ('tall', 'JJ'), ('brick', 'NN'), ('wall', 'NN')]

    Answer: Here are the grammatical classifications:
    [('The', 'DT'), ('black', 'JJ'), ('cat', 'NN'), ('jumped', 'VBD'), ('over', 'IN'), ('the', 'DT'), ('tall', 'JJ'), ('brick', 'NN'), ('wall', 'NN')]
    """.strip()

    GrammarAgent = Agent(client, system_prompt)
    response = GrammarAgent(f"Analyze this sentence: {sentence}")
    GrammarAgent.reset()

    # Loop principal de interação
    while True:
        if "PAUSE" in response:
            # Extrai e executa ações dinamicamente
            action_line = next((line for line in response.split('\n') if line.startswith("Action:")), None)
            if action_line:
                action = action_line.split("Action:")[1].strip()
                if action.startswith("class_words("):
                    # Executa a análise linguística
                    frase = action[len("class_words("):-1].strip()
                    if not frase: 
                        frase = sentence
                    observation = class_words(frase)
                    response = GrammarAgent(f"Observation: {observation}")
                else:
                    response = GrammarAgent("Observation: Unknown action")
            else:
                response = GrammarAgent("Observation: No action specified")
        
        elif "Answer:" in response:
            # Finaliza quando obtém uma resposta completa
            return response.split("Answer:")[1].strip()
        
        else:
            response = GrammarAgent()

### Exemplo de uso

In [8]:
if __name__ == "__main__":
    input_sentence = "The black cat jumps over the tall brick wall, falling on a dog."
    result = classifie_sentence(input_sentence)
    print(f"Analysis result for '{input_sentence}':")
    print(result)

Analysis result for 'The black cat jumps over the tall brick wall, falling on a dog.':
Here are the grammatical classifications: [('The', 'DT), ('black', 'JJ'), ('cat', 'NN'), ('jumps', 'NNS'), ('over', 'IN'), ('the', 'DT'), ('tall', 'JJ'), ('brick', 'NN'), ('wall', 'NN'), (',', ','), ('falling', 'VBG'), ('on', 'IN'), ('a', 'DT'), ('dog', 'NN')]
